### Data Kelayakan Kartu Kredit: Faktor Penentu

Final Project Mechine Learning Indonesia AI

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

ImportError: cannot import name 'RandomOverSampler' from 'imblearn.under_sampling' (D:\LUTFI\Anacnda\envs\streamlit\lib\site-packages\imblearn\under_sampling\__init__.py)

In [ ]:
#Membaca data
data = pd.read_csv('dataset.csv')

In [ ]:
#Mengganti nama data menjadi df_data
df_data = data.copy()

#dimensi data
print(df_data.shape)

In [ ]:
df_data.head()

In [ ]:
#Ringkasan dataset
df_data.info()

*data integer : 12 kolom* <br>
*data float : 3 kolom* <br>
*data object : 5 kolom*

In [ ]:
#Melihat data yang duplikat 
df_data.duplicated().sum()

*data tidak memiliki data yang duplikat*

In [ ]:
#Mengecek nilai kosong
df_data.isnull().mean()*100

*data tidak memiliki data yang kosong*

In [ ]:
#Membuang kolom ID 
df_data.drop(columns = ['ID'], axis=1, inplace=True)
df_data

In [ ]:
#Mengecek jumlah kategori disetiap kolom kategori

for i in df_data.columns:
    if df_data[str(i)].dtype =='O':

        print(i)
        print(len(df_data[str(i)].drop_duplicates()))

        if len(df_data[str(i)].drop_duplicates()) > 10:
            print('Hapus Kolom = ', str(i))
            df_data.drop(str(i), axis=1, inplace=True)

            print('')

In [ ]:
#Ringkasan Data
df_data.describe(include = 'O')

In [ ]:
#Mengecek income_type future
np.unique(df_data['Income_type'])

In [ ]:
sns.countplot(x='Income_type', data=df_data)
plt.figure(figsize=(20,20))
plt.show()


In [ ]:
#Mengecek Family_Status future
np.unique(df_data['Family_status'])

In [ ]:
sns.countplot(x='Family_status', data=df_data)
plt.figure(figsize=(20,20))
plt.show()

In [ ]:
#Melihat proporsi data target
print(df_data['Target'].value_counts()/len(df_data['Target']))

*data yang bernilai 0 867855* <br>
*data yang bernilai 1 132145* <br>
data ini menunjukkan ketimpangan antara 0 dan 1

In [ ]:
#visualisasi 
plt.figure(figsize=(8,6))
sns.countplot(x = df_data['Target']);
plt.show()

*sangat jomplang sekali data 0 dan 1*

In [ ]:
#Melihat kolom kategori yang bertipe object

kategori = [var for var in df_data.columns if df_data[var].dtype == 'O']

print(kategori)

In [ ]:
#dirubah menjadi tipe category
for i in kategori:
    df_data[i] = df_data[i].astype('category')

In [ ]:
df_data.info()

In [ ]:
#Ringkasan data kategory
df_data.describe(include='category')

In [ ]:
target = 'Target'

In [ ]:
#Mengelompokkan data kategory dan data numerik
feature_kategory = [var for var in df_data.columns if df_data[var].dtype == 'category' and var!='Target']
feature_numerik = [var for var in df_data.columns if df_data[var].dtype != 'category' and var!='Target']

In [ ]:
print(feature_kategory)
print(feature_numerik)

In [ ]:
#Melihat pairplot
sns.pairplot(df_data)

### Menentukan Variabel X dan Variabel y

In [ ]:
df_data.shape

In [ ]:
#Membagi data 
X = df_data.iloc[:, df_data.columns != 'Target']
y = df_data['Target']

In [ ]:
X.head()

In [ ]:
y.head()

### Spliting Dataset

In [ ]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = 0)

# Melihat dimensi training set dan test set
print((X_train.shape), (X_test.shape))

### Balancing Dataset

In [ ]:
# Melihat proporsi yang baru
print(y_train.value_counts()/len(y_train))

In [ ]:
# Melihat dimensi data
print(X_train.shape)
print(y_train.shape)

In [ ]:
# Reset index X_train dan X_test
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
# Membuat Pipeline untuk preprocessing
preprocessor_numerik = Pipeline([
    ('imputasi', SimpleImputer(strategy='median')),
    ('scaling', StandardScaler())
])

preprocessor_kategori = Pipeline([
    ('imputasi', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('encoding', OneHotEncoder(drop='first', handle_unknown='ignore'))         
])

# Menggabungkan kedua pipeline di atas
preprocessor = ColumnTransformer([
    ('preprocessing numerik', preprocessor_numerik, feature_numerik),
    ('preprocessing kategori', preprocessor_kategori, feature_kategory)
])

In [ ]:
# Model regresi logistik
logreg = LogisticRegression(random_state=123, max_iter=1000)

# Pipeline model regresi logistik
pipe_logreg = ImbPipeline([
    ('preprocessing', preprocessor),
    ('smote', SMOTE(random_state=123)),
    ('logreg', logreg)
])

In [ ]:
#fiting
pipe_logreg.fit(X_train, y_train)

In [ ]:
# Mencoba memprediksi test set
pred_test = pipe_logreg.predict(X_test)

# Merubahnya ke format Dataframe
pred_test = pd.DataFrame(pred_test, columns=['Target'])

# Melihat tmapilan y_pred
pred_test.head()

In [ ]:
confuse_log = confusion_matrix(y_test, pred_test)
confuse_log

In [ ]:
# Melihat confusion matrix dengan ConfusionMatrixDisplay()
vis_cm = ConfusionMatrixDisplay(confusion_matrix = confuse_log,
                                display_labels = pipe_logreg.classes_)
vis_cm.plot(cmap=plt.cm.Blues)

In [ ]:
# Melihat ringkasan evaluasi
print(classification_report(y_test,pred_test))

In [ ]:
# Menghitung metrics klasifikasi satu per satu
print('Nilai akurasi: {:.2f}'.format(accuracy_score(y_test, pred_test)))
print('Nilai presisi: {:.2f}'.format(precision_score(y_test, pred_test)))
print('Nilai recall: {:.2f}'.format(recall_score(y_test, pred_test)))
print('Nilai f1: {:.2f}'.format(f1_score(y_test, pred_test)))

In [ ]:
# Model regresi logistik
logreg = LogisticRegression(max_iter=1000)

# Pipeline model regresi logistik
pipe_logreg = Pipeline([
    ('preprocessing', preprocessor),
    ('logreg', logreg)
])

# Hyperparameter tuning Logistic Regression
param_logreg = {'logreg__penalty': ['l2'],
                'logreg__C': [0.01, 0.09, 0.1, 0.125, 1, 1.5],
                'logreg__solver' : ['liblinear', 'newton-cg', 'sag', 'saga', 'lbfgs']
               }

In [ ]:
# Model SVM
svm = SVC(kernel = 'rbf', random_state = 0)

# Pipeline model SVM
pipe_svm = Pipeline([
    ('preprocessing', preprocessor),
    ('svm', svm)
])

# Hyperparameter tuning SVM
param_svm = {'svm__C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 1],
             'svm__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
             'svm__degree': [1, 2, 3, 4, 5],
             'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid']
            }

In [ ]:
# Model RF
rf = RandomForestClassifier(random_state=0, n_jobs=-1)

# Pipeline model RF
pipe_rf = Pipeline([
    ('preprocessing', preprocessor),
    ('rf', rf),
])

# Hyperparameter tuning RF
param_rf = {'rf__n_estimators': [200, 250, 300],
            'rf__criterion': ['gini', 'entropy', 'log_loss'],
            'rf__max_depth': [1, 2, 3, 4],
            'rf__min_samples_split': [2, 3],
            'rf__min_samples_leaf': [1, 2, 3],
            'rf__warm_start': [True]
            }

In [ ]:
# Model AdaBoost
ada = AdaBoostClassifier(random_state=0)

# Pipeline model AdaBoost
pipe_ada = Pipeline([
    ('preprocessing', preprocessor),
    ('ada', ada),
])

# Hyperparameter tuning AdaBoost
param_ada = {'ada__n_estimators': [10, 20, 30, 50, 70],
             'ada__learning_rate': [0.3, 0.4, 0.5, 1, 1.1],
             'ada__algorithm': ['SAMME', 'SAMME.R'],
             }

In [ ]:
# Model XGBoost
xgb = XGBClassifier(random_state=0, n_jobs=-1)

# Pipeline model XGBoost
pipe_xgb = Pipeline([
    ('preprocessing', preprocessor),
    ('xgb', xgb),
])

param_xgb = {
    'xgb__n_estimators': [200, 300, 400, 500, 600],
    'xgb__max_depth': [3, 5, 7, 9, 11],
    'xgb__learning_rate': [0.05, 0.1, 0.2, 0.3, 0.4],
    'xgb__gamma': [0, 0.1, 0.2, 0.3, 0.4],
}

### Pipeline dan Cross Validation

In [ ]:
# Model training dengan cross validation
daftar_model = [pipe_logreg, pipe_svm,  pipe_rf, pipe_ada, pipe_xgb]
daftar_nama_model = ['logreg', 'svm', 'rf', 'ada', 'xgb']
mean_akurasi = []
mean_auc = []
std_akurasi = []
std_auc = []
test_score_akurasi = []
test_score_auc = []

# Kita setting agar proses cross validasi dilakukan dengan sama rata (stratified) untuk pembagian kategori labelnya
skf = StratifiedKFold(n_splits=3, random_state=0, shuffle=True)
cv = skf

# Looping untuk setiap model yang sudah disiapkan 
for i in daftar_model:
    
    # Melakukan cross validation dan menggunakan kriteria berdasarkan skor akurasi
    cv_akurasi = cross_val_score(i, X_train, y_train, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)  # kita kalikan dengan -1 karena scoring menggunakan nilai negatif
    cv_auc = cross_val_score(i, X_train, y_train, cv=cv, scoring='roc_auc', verbose=1, n_jobs=-1)
    
    # Menghitung nilai rata-rata akurasi dan menambahkannya ke variabel mean_akurasi
    mean_akurasi.append(round(cv_akurasi.mean(),2))   # round(nilai,2) untuk membulatkan nilai 2 angka di belakang koma
    mean_auc.append(round(cv_auc.mean(),2))

    # Menghitung nilai standar deviasi akurasi dan menambahkannya ke variabel std_akurasi
    std_akurasi.append(round(cv_akurasi.std(),2))
    std_auc.append(round(cv_auc.std(),2))
    
    # Melakukan fitting training set kemudian melakukan prediksi  di test set
    i.fit(X_train, y_train)
    i_predict = i.predict(X_test)
    
    # Menghitung nilai rata-rata akurasi di test set dan menambahkannya ke variabel test_score_akurasi
    test_score_akurasi.append(round(accuracy_score(y_test, i_predict),2))
    test_score_auc.append(round(roc_auc_score(y_test, i_predict),2))

In [ ]:
# Membuat DataFrame
cv_akurasi = pd.DataFrame({'model':daftar_nama_model, 'Train_Mean':mean_akurasi, 'std':std_akurasi, 'Test_Score':test_score_akurasi})
cv_auc = pd.DataFrame({'model':daftar_nama_model, 'Train_Mean':mean_auc, 'std':std_auc, 'Test_Score':test_score_auc})

In [ ]:
# Melihat hasil cross validation berdasarkan akurasi
cv_akurasi

In [ ]:
# Melihat hasil cross validation berdasarkan akurasi
cv_auc

In [ ]:
# Mengurutkan skor dari kecil ke besar berdasarkan skor akurasi
cv_akurasi_urut = cv_akurasi.sort_values(by=['Train_Mean', 'Test_Score'], ascending=False,  ignore_index=True)
cv_akurasi_urut

In [ ]:
# Mengurutkan skor dari kecil ke besar berdasarkan skor AUC
cv_auc_urut = cv_auc.sort_values(by=['Train_Mean', 'Test_Score'], ascending=False,  ignore_index=True)
cv_auc_urut

### Hyperparameter Tuning

In [ ]:
# Model training dengan RandomSearchCV
daftar_model = [pipe_logreg, pipe_svm,  pipe_rf, pipe_ada, pipe_xgb]
daftar_nama_model = ['logreg', 'svm', 'rf', 'ada', 'xgb']
daftar_param_model = [param_logreg,  param_svm, param_rf, param_ada, param_xgb]

# Nilai yang akan diisikan
akurasi_tuning = []
akurasi_tuning_test = []
auc_tuning = []
auc_tuning_test = []
best_param = []
best_estimator = []

for i in range(len(daftar_model)):
    
    # Menjalankan RandomizedSearchCV
    model_random_cv = RandomizedSearchCV(
        daftar_model[i],
        daftar_param_model[i],
        cv=5,
        scoring='accuracy',  # nilai akurasi digunakan sebagai dasar penentuan peringkat parameter terbaik dan parameter scoring ini bisa diganti yang lain
        verbose=1,
        n_jobs=-1
        )
    
    # Fitting ke training set
    model_random_cv.fit(X_train,y_train)
    
    # Mencoba memprediksi training dan test set setelah fitting di training set, kemudian dikemas dalam format DataFrame
    pred_train = pd.DataFrame(model_random_cv.predict(X_train), columns=['Eligible'])
    pred_test = pd.DataFrame(model_random_cv.predict(X_test), columns=['Eligible'])

    # Mencatat skor MAE training dan test set
    akurasi_tuning.append(accuracy_score(y_train, pred_train))
    akurasi_tuning_test.append(accuracy_score(y_test, pred_test))

    # Mencatat skor R2 training dan test set
    auc_tuning.append(roc_auc_score(y_train, pred_train))
    auc_tuning_test.append(roc_auc_score(y_test, pred_test))   
    
    # Mencatat parameter terbaik di setiap model
    best_param.append(model_random_cv.best_params_)
    
    # Merekam settingan modelnya
    best_estimator.append(model_random_cv.best_estimator_)

In [ ]:
# Model training dengan GridSearchCV
daftar_model = [pipe_logreg, pipe_svm,  pipe_rf, pipe_ada, pipe_xgb]
daftar_nama_model = ['logreg', 'svm', 'rf', 'ada', 'xgb']
daftar_param_model = [param_logreg,  param_svm, param_rf, param_ada, param_xgb]

# Nilai yang akan diisikan
akurasi_tuning = []
akurasi_tuning_test = []
auc_tuning = []
auc_tuning_test = []
best_param = []
best_estimator = []

for i in range(len(daftar_model)):
    
    # Menjalankan GridSearchCV
    model_grid_cv = GridSearchCV(
        daftar_model[i],
        daftar_param_model[i],
        cv=3,
        scoring='accuracy',  # nilai akurasi digunakan sebagai dasar penentuan peringkat parameter terbaik dan parameter scoring ini bisa diganti yang lain
        verbose=1,
        n_jobs=-1
        )
    
    # Fitting ke training set
    model_grid_cv.fit(X_train,y_train)
    
    # Mencoba memprediksi training dan test set setelah fitting di training set, kemudian dikemas dalam format DataFrame
    pred_train = pd.DataFrame(model_grid_cv.predict(X_train), columns=['Eligible'])
    pred_test = pd.DataFrame(model_grid_cv.predict(X_test), columns=['Eligible'])

    # Mencatat skor MAE training dan test set 
    akurasi_tuning.append(accuracy_score(y_train, pred_train))
    akurasi_tuning_test.append(accuracy_score(y_test, pred_test))

    # Mencatat skor R2 training dan test set
    auc_tuning.append(roc_auc_score(y_train, pred_train))
    auc_tuning_test.append(roc_auc_score(y_test, pred_test))   
    
    # Mencatat parameter terbaik di setiap model
    best_param.append(model_grid_cv.best_params_)
    
    # Merekam settingan modelnya
    best_estimator.append(model_grid_cv.best_estimator_)

In [ ]:
# Membuat DataFrame sekaligus kita urutkan
grid_akurasi = pd.DataFrame({'model':daftar_nama_model, 'Training':akurasi_tuning, 'Testing':akurasi_tuning_test})
grid_akurasi_urut = grid_akurasi.sort_values(by='Testing', ascending=False,  ignore_index=True)

grid_auc = pd.DataFrame({'model':daftar_nama_model, 'Training':auc_tuning, 'Testing':auc_tuning_test})
grid_auc_urut = grid_auc.sort_values(by='Testing', ascending=False, ignore_index=True)

In [ ]:
# Melihat performa tuning berdasarkan MAE
grid_akurasi_urut

In [ ]:
# Melihat performa tuning berdasarkan R2
grid_auc_urut

In [ ]:
# Parameter terbaik dari model terbaik
grid_param = pd.DataFrame({'model':daftar_nama_model, 'Param':best_param})
grid_param

In [ ]:
# Estimator terbaik dari model terbaik
grid_estimator = pd.DataFrame({'model':daftar_nama_model, 'Param':best_estimator})
grid_estimator

In [ ]:
# Kita gabungkan semua sebagai 'model_best'
model_best = pd.DataFrame({'model':daftar_nama_model, 'Param':best_estimator, 'Testing':akurasi_tuning_test})
model_best = model_best.sort_values(by='Testing', ascending=False, ignore_index=True)
model_best

### model Terbaik

In [ ]:
# Fitting ke dataset utuh
model_final = model_best['Param'][2].fit(X, y)

In [ ]:
# Melihat spesifikasi model_final
model_final

### Melihat *feature importance* dari model terbaik.

In [ ]:
# # Menghitung importances dari pipeline adaboost
# importances = model_final[1].feature_importances_
# importances = pd.Series(importances, index=model_final['preprocessing'].get_feature_names_out())

In [ ]:
# # Melakukan plotting
# fig, ax = plt.subplots(figsize = (8,10))
# importances.sort_values(ascending=False).plot.bar(ax=ax)
# ax.set_title("Plot Feature Importance Setiap Feature")
# ax.set_ylabel("Mean decrease in impurity")
# fig.tight_layout()

In [ ]:
# importances = model_final[1].feature_importances_
# importances = pd.Series(importances, index=model_final['preprocessing'].get_feature_names_out())

# # Filter importances to keep only non-zero values
# non_zero_importances = importances[importances > 0]

# # Create a new DataFrame from the filtered series
# important_features_df = pd.DataFrame({
#     'Feature': non_zero_importances.index,
#     'Importance': non_zero_importances.values
# })

# important_features_df['Feature'] = important_features_df['Feature'].str.replace('preprocessing numerik__', '', regex=False)
# important_features_df['Feature'] = important_features_df['Feature'].str.replace('preprocessing kategori__', '', regex=False)

# # Optionally, sort the DataFrame by importance
# important_features_df = important_features_df.sort_values(by='Importance', ascending=False)

# # Display the new DataFrame
# important_features_df

In [ ]:
# transformed_data = preprocessor.fit_transform(X_train)

# encoded_cat_columns = preprocessor.named_transformers_['preprocessing kategori'].get_feature_names_out(feature_kategory)

# transformed_categorical_df = pd.DataFrame(transformed_data[:, len(feature_numerik):], columns=encoded_cat_columns)

# transformed_df = pd.concat([pd.DataFrame(transformed_data[:, :len(feature_numerik)], columns=feature_numerik), transformed_categorical_df], axis=1)

# df_baru = transformed_df[important_features_df['Feature']]

In [ ]:
# Memprediksi df_test
hasil = model_final.predict(df_data)
hasil

In [ ]:
df_test = pd.read_csv('dataset.csv')
df_test.head()

In [ ]:
# Memprediksi df_test
hasil = model_final.predict(df_test)
hasil

In [ ]:
# # Keputusan
# for i in range(len(hasil)):
#     if hasil[i] == 1:
#         print('Data pelanggan',df_test['ID'][i],'= Layak')
#     else:
#         print('Data pelanggan',df_test['ID'][i],'= Tidak Layak')

In [ ]:
# Library untuk menyimpan model
import pickle

# Menyimpan model dengan nama 'model_regresi_terbaik.pkl'
pickle.dump(model_final, open('model_creditcard.pkl', 'wb'))

# Load lagi model yang sudah disimpan dan buka sebagai 'best_model'
best_model = pickle.load(open('model_creditcard.pkl', 'rb'))

In [ ]:
# Mencoba melakukan prediksi X_test
prediksi = best_model.predict(df_test)

# Melihat hasil prediksi
prediksi